# Comparing XAI


Using Jaccard Similarity to compare features from SHAP and LIME

### Setup
Need to combine both dfs

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import dtale
import matplotlib.pyplot as plt
import seaborn as sns


from master_thesis.config import PROCESSED_DATA_DIR, load_dataframe_from_pickle

2025-04-22 10:30:38.526 | INFO     | master_thesis.config:<module>:12 - PROJ_ROOT path is: /home/takosaga/Projects/master_thesis


In [2]:
full_hatespeech_df = load_dataframe_from_pickle(
    PROCESSED_DATA_DIR / "annotated_and_targeted_hatespeech.pkl"
)

full_hatespeech_df.head(5)

,id_new,text,extracted_target,label_hatespeech_binary_offensive_not_included,label_hatespeech_binary_offensive_included,label_normal_offensive_hatespeech,platform,original_dataset_title,original_id,original_label,original_target
0,1,i dont think im getting my baby them white 9 h...,{none},not_hatespeech,normal,normal,twitter,HateXplain,1179055004553900032,normal,[None]
1,2,we cannot continue calling ourselves feminists...,{none},not_hatespeech,normal,normal,twitter,HateXplain,1179063826874032128,normal,[None]
2,3,nawt yall niggers ignoring me,{race},not_hatespeech,normal,normal,twitter,HateXplain,1178793830532956161,normal,[African]
3,4,<user> i am bit confused coz chinese ppl can n...,{race},hatespeech,hatespeech/offensive,hatespeech,twitter,HateXplain,1179088797964763136,hatespeech,[Asian]
4,5,this bitch in whataburger eating a burger with...,"{race, gender}",hatespeech,hatespeech/offensive,hatespeech,twitter,HateXplain,1179085312976445440,hatespeech,"[Caucasian, Women]"


In [3]:
full_hatespeech_df.columns.to_list()

['id_new',
 'text',
 'extracted_target',
 'label_hatespeech_binary_offensive_not_included',
 'label_hatespeech_binary_offensive_included',
 'label_normal_offensive_hatespeech',
 'platform',
 'original_dataset_title',
 'original_id',
 'original_label',
 'original_target']

In [4]:
columns_to_keep = ['id_new', 'text', 'extracted_target','label_hatespeech_binary_offensive_included','platform']
adjusted_hatespeech_df = full_hatespeech_df[columns_to_keep]


In [5]:
adjusted_hatespeech_df.head(5)

,id_new,text,extracted_target,label_hatespeech_binary_offensive_included,platform
0,1,i dont think im getting my baby them white 9 h...,{none},normal,twitter
1,2,we cannot continue calling ourselves feminists...,{none},normal,twitter
2,3,nawt yall niggers ignoring me,{race},normal,twitter
3,4,<user> i am bit confused coz chinese ppl can n...,{race},hatespeech/offensive,twitter
4,5,this bitch in whataburger eating a burger with...,"{race, gender}",hatespeech/offensive,twitter


In [6]:
adjusted_hatespeech_df['is_hatespeech'] = adjusted_hatespeech_df['label_hatespeech_binary_offensive_included'].apply(lambda x: 1 if x == 'hatespeech/offensive' else 0)
adjusted_hatespeech_df['id'] = adjusted_hatespeech_df['id_new']

/tmp/ipykernel_48865/2421778535.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_48865/2421778535.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [7]:
adjusted_hatespeech_df.drop(columns=['label_hatespeech_binary_offensive_included','id_new'], inplace=True)

/tmp/ipykernel_48865/3016290075.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [8]:
adjusted_hatespeech_df.head(5)

,text,extracted_target,platform,is_hatespeech,id
0,i dont think im getting my baby them white 9 h...,{none},twitter,0,1
1,we cannot continue calling ourselves feminists...,{none},twitter,0,2
2,nawt yall niggers ignoring me,{race},twitter,0,3
3,<user> i am bit confused coz chinese ppl can n...,{race},twitter,1,4
4,this bitch in whataburger eating a burger with...,"{race, gender}",twitter,1,5


In [9]:
XAI_df = load_dataframe_from_pickle(
    PROCESSED_DATA_DIR / "models_and_XAI_applied.pkl"
)

XAI_df.head(5)

,text,error,cardiff_label,cardiff_score,fb_label,fb_score,lime_cardiff,lime_fb,shap_cardiff,shap_fb,source_file
0,i dont think im getting my baby them white 9 h...,NaN,NOT-HATE,0.992369,hate,0.687105,"and: 0.04, my: 0.04, white: -0.03, 9: -0.03","white: -0.19, touched: -0.12, dont: -0.11, j: ...","i : 0.002, getting : 0.001, my : 0.001, baby :...",": 0.014, i : 0.014, dont : -0.008, think : -0....",df_1
1,we cannot continue calling ourselves feminists...,NaN,NOT-HATE,0.997838,nothate,0.919145,"trans: 0.00, gender: 0.00, addressed: 0.00, fe...","and: 0.13, trans: 0.11, lesbian: 0.11, a: -0.10","calling : 0.013, ourselves : 0.014, feminists ...",": -0.014, we : -0.014, cannot : -0.029, contin...",df_1
2,nawt yall niggers ignoring me,NaN,HATE,0.935527,hate,0.999480,"niggers: -0.90, nawt: -0.09, me: 0.07, yall: 0.07","niggers: -0.96, ignoring: -0.02, nawt: -0.02, ...","n: 0.010, aw: 0.006, t : 0.007, y: 0.003, all ...","n: 0.030, aw: -0.062, t : -0.040, y: 0.129, al...",df_1
3,<user> i am bit confused coz chinese ppl can n...,NaN,HATE,0.997049,hate,0.779210,"chong: -0.44, ching: -0.29, chinese: -0.08, pa...","ching: -0.26, this: -0.24, chong: -0.23, confu...",": -0.005, <: -0.005, user: -0.005, > : -0.005,...",": 0.016, <: 0.016, user: 0.016, > : 0.016, i :...",df_1
4,this bitch in whataburger eating a burger with...,NaN,HATE,0.990916,hate,0.999297,"bitches: -0.44, bitch: -0.28, hate: -0.18, eat...","bitch: -0.39, bitches: -0.30, holding: -0.05, ...","bitch : -0.338, in : -0.145, urger : 0.007, ea...","this : 0.007, bitch : -0.478, in : -0.004, wha...",df_1


Checking for errors

In [10]:
XAI_df['error'].value_counts()

error
Caught an exception: The expanded size of the tensor (1062) must match the existing size (514) at non-singleton dimension 1.  Target sizes: [1, 1062].  Tensor sizes: [1, 514]    1
Name: count, dtype: int64

In [11]:
non_nan_mask = XAI_df['error'].notna()

In [12]:
XAI_df.drop(XAI_df[non_nan_mask].index, inplace=True)

In [13]:
XAI_df['error'].value_counts()

Series([], Name: count, dtype: int64)

In [14]:
XAI_df.columns.to_list()


['text',
 'error',
 'cardiff_label',
 'cardiff_score',
 'fb_label',
 'fb_score',
 'lime_cardiff',
 'lime_fb',
 'shap_cardiff',
 'shap_fb',
 'source_file']

In [17]:
columns_to_keep_2 = ['text', 'cardiff_label', 'cardiff_score', 'shap_cardiff', 'lime_cardiff', 'fb_label','fb_score', 'shap_fb', 'lime_fb']

In [18]:
XAI_df = XAI_df[columns_to_keep_2]

In [20]:
XAI_df.head(5)

,text,cardiff_label,cardiff_score,shap_cardiff,lime_cardiff,fb_label,fb_score,shap_fb,lime_fb
0,i dont think im getting my baby them white 9 h...,NOT-HATE,0.992369,"i : 0.002, getting : 0.001, my : 0.001, baby :...","and: 0.04, my: 0.04, white: -0.03, 9: -0.03",hate,0.687105,": 0.014, i : 0.014, dont : -0.008, think : -0....","white: -0.19, touched: -0.12, dont: -0.11, j: ..."
1,we cannot continue calling ourselves feminists...,NOT-HATE,0.997838,"calling : 0.013, ourselves : 0.014, feminists ...","trans: 0.00, gender: 0.00, addressed: 0.00, fe...",nothate,0.919145,": -0.014, we : -0.014, cannot : -0.029, contin...","and: 0.13, trans: 0.11, lesbian: 0.11, a: -0.10"
2,nawt yall niggers ignoring me,HATE,0.935527,"n: 0.010, aw: 0.006, t : 0.007, y: 0.003, all ...","niggers: -0.90, nawt: -0.09, me: 0.07, yall: 0.07",hate,0.999480,"n: 0.030, aw: -0.062, t : -0.040, y: 0.129, al...","niggers: -0.96, ignoring: -0.02, nawt: -0.02, ..."
3,<user> i am bit confused coz chinese ppl can n...,HATE,0.997049,": -0.005, <: -0.005, user: -0.005, > : -0.005,...","chong: -0.44, ching: -0.29, chinese: -0.08, pa...",hate,0.779210,": 0.016, <: 0.016, user: 0.016, > : 0.016, i :...","ching: -0.26, this: -0.24, chong: -0.23, confu..."
4,this bitch in whataburger eating a burger with...,HATE,0.990916,"bitch : -0.338, in : -0.145, urger : 0.007, ea...","bitches: -0.44, bitch: -0.28, hate: -0.18, eat...",hate,0.999297,"this : 0.007, bitch : -0.478, in : -0.004, wha...","bitch: -0.39, bitches: -0.30, holding: -0.05, ..."


In [22]:
XAI_df['cardiff_hatespeech'] = XAI_df['cardiff_label'].apply(lambda x: 1 if x == 'HATE' else 0)

/tmp/ipykernel_48865/3440690672.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [23]:
XAI_df['fb_hatespeech'] = XAI_df['cardiff_label'].apply(lambda x: 1 if x == 'hate' else 0)

In [24]:
XAI_df.drop(columns=['cardiff_label','fb_label'], inplace=True)

In [25]:
XAI_df.head(5)

,text,cardiff_score,shap_cardiff,lime_cardiff,fb_score,shap_fb,lime_fb,cardiff_hatespeech,fb_hatespeech
0,i dont think im getting my baby them white 9 h...,0.992369,"i : 0.002, getting : 0.001, my : 0.001, baby :...","and: 0.04, my: 0.04, white: -0.03, 9: -0.03",0.687105,": 0.014, i : 0.014, dont : -0.008, think : -0....","white: -0.19, touched: -0.12, dont: -0.11, j: ...",0,0
1,we cannot continue calling ourselves feminists...,0.997838,"calling : 0.013, ourselves : 0.014, feminists ...","trans: 0.00, gender: 0.00, addressed: 0.00, fe...",0.919145,": -0.014, we : -0.014, cannot : -0.029, contin...","and: 0.13, trans: 0.11, lesbian: 0.11, a: -0.10",0,0
2,nawt yall niggers ignoring me,0.935527,"n: 0.010, aw: 0.006, t : 0.007, y: 0.003, all ...","niggers: -0.90, nawt: -0.09, me: 0.07, yall: 0.07",0.999480,"n: 0.030, aw: -0.062, t : -0.040, y: 0.129, al...","niggers: -0.96, ignoring: -0.02, nawt: -0.02, ...",1,0
3,<user> i am bit confused coz chinese ppl can n...,0.997049,": -0.005, <: -0.005, user: -0.005, > : -0.005,...","chong: -0.44, ching: -0.29, chinese: -0.08, pa...",0.779210,": 0.016, <: 0.016, user: 0.016, > : 0.016, i :...","ching: -0.26, this: -0.24, chong: -0.23, confu...",1,0
4,this bitch in whataburger eating a burger with...,0.990916,"bitch : -0.338, in : -0.145, urger : 0.007, ea...","bitches: -0.44, bitch: -0.28, hate: -0.18, eat...",0.999297,"this : 0.007, bitch : -0.478, in : -0.004, wha...","bitch: -0.39, bitches: -0.30, holding: -0.05, ...",1,0


In [27]:
full_df = pd.merge(adjusted_hatespeech_df,XAI_df, on='text', how='inner')

In [28]:
full_df.head()

,text,extracted_target,platform,is_hatespeech,id,cardiff_score,shap_cardiff,lime_cardiff,fb_score,shap_fb,lime_fb,cardiff_hatespeech,fb_hatespeech
0,i dont think im getting my baby them white 9 h...,{none},twitter,0,1,0.992369,"i : 0.002, getting : 0.001, my : 0.001, baby :...","and: 0.04, my: 0.04, white: -0.03, 9: -0.03",0.687105,": 0.014, i : 0.014, dont : -0.008, think : -0....","white: -0.19, touched: -0.12, dont: -0.11, j: ...",0,0
1,we cannot continue calling ourselves feminists...,{none},twitter,0,2,0.997838,"calling : 0.013, ourselves : 0.014, feminists ...","trans: 0.00, gender: 0.00, addressed: 0.00, fe...",0.919145,": -0.014, we : -0.014, cannot : -0.029, contin...","and: 0.13, trans: 0.11, lesbian: 0.11, a: -0.10",0,0
2,nawt yall niggers ignoring me,{race},twitter,0,3,0.935527,"n: 0.010, aw: 0.006, t : 0.007, y: 0.003, all ...","niggers: -0.90, nawt: -0.09, me: 0.07, yall: 0.07",0.999480,"n: 0.030, aw: -0.062, t : -0.040, y: 0.129, al...","niggers: -0.96, ignoring: -0.02, nawt: -0.02, ...",1,0
3,<user> i am bit confused coz chinese ppl can n...,{race},twitter,1,4,0.997049,": -0.005, <: -0.005, user: -0.005, > : -0.005,...","chong: -0.44, ching: -0.29, chinese: -0.08, pa...",0.779210,": 0.016, <: 0.016, user: 0.016, > : 0.016, i :...","ching: -0.26, this: -0.24, chong: -0.23, confu...",1,0
4,this bitch in whataburger eating a burger with...,"{race, gender}",twitter,1,5,0.990916,"bitch : -0.338, in : -0.145, urger : 0.007, ea...","bitches: -0.44, bitch: -0.28, hate: -0.18, eat...",0.999297,"this : 0.007, bitch : -0.478, in : -0.004, wha...","bitch: -0.39, bitches: -0.30, holding: -0.05, ...",1,0
